### Imports

In [3]:
import requests
import bs4
import openai
import os

### Get API key

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")

### Setup dictionary for foreign language news-url

In [42]:
country = input("What country are you interested in for news?")

In [43]:
country_newspapers = {'Spain': 'https://elpais.com/', 
                      'France' : 'https://www.lemonde.fr/'}

In [44]:
country_newspapers['Spain']

'https://elpais.com/'

In [45]:
url = country_newspapers['France']

In [46]:
result = requests.get(url)

In [47]:
#result.text

In [48]:
tag = 'article__title-label'

#### Extract text using beautiful soup

In [49]:
soup = bs4.BeautifulSoup(result.text,'lxml')

In [50]:
for item in soup.select('.article__title')[:4]:
    print(item.getText())

Daniel Kretinsky, nouveau poids lourd de l’emploi en France
Microsoft-Activision Blizzard : « La revanche de la Big Tech sur les “bidenomics” »
Quelle prestation compensatoire pour les divorces homosexuels ?
« Le changement climatique renforce les inégalités Nord-Sud dans une injustice insupportable »


### Add tag to the dictionary

In [51]:
country_newspapers = {'Spain': ('https://elpais.com/', '.c_t'),
                      'France': ('https://www.lemonde.fr/', '.article__title')}

### Prompt creation

In [52]:
def create_prompt():
    country = input("What country would you like a news summary for?")

    try:
        url, tag = country_newspapers[country]
    except:
        print("Not Supported")
        return 
    
    results = requests.get(url)
    soup = bs4.BeautifulSoup(results.text, 'lxml')

    country_headlines=''
    for item in soup.select(tag)[:3]:
        country_headlines +=item.getText() +'\n'

    prompt = "Detect the language of the news headlines below, then translate a summary of the headlines to English in a conversational tone. \n"

    return prompt + country_headlines

In [66]:
prompt = create_prompt()

In [68]:
print(prompt)

Detect the language of the news headlines below, then translate a summary of the headlines to English in a conversational tone. 
DeSantis reconoce que Biden ganó las elecciones de 2020 y enmienda la plana a Trump: “Por supuesto que perdió” 
Argentina comienza a elegir a su próximo presidente
El primer año de Gustavo Petro en cinco fotos: un viaje del optimismo al pesimismo



### OpenAI API Call

In [69]:
response = openai.Completion.create(model='text-davinci-003',
                                    prompt = prompt,
                                    max_tokens=256,  
                                    temperature = 0.1,
                                    top_p=1.0)

### Results

In [72]:
print(response['choices'][0]['text'])


The first headline is in Spanish. It translates to: Governor DeSantis acknowledges that Biden won the 2020 elections and corrects Trump: "Of course he lost". 

The second headline is in Spanish. It translates to: Argentina begins to elect its next president. 

The third headline is in Spanish. It translates to: Gustavo Petro's first year in five photos: a journey from optimism to pessimism.
